# Dataset to SQL

In [120]:
import pandas as pd
import mysql.connector

In [121]:
%pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [122]:
import mysql.connector
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='markcelemen1',
    database='superstore_sales'
)
cursor = connection.cursor() 
cursor

In [123]:
superstore_df = pd.read_csv('cleaned_superstore.csv')
superstore_df = superstore_df.where(pd.notnull(superstore_df), None)
superstore_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_id            9994 non-null   int64  
 1   order_id          9994 non-null   object 
 2   order_date        9994 non-null   object 
 3   ship_date         9994 non-null   object 
 4   ship_mode         9994 non-null   object 
 5   customer_id       9994 non-null   object 
 6   customer_name     9994 non-null   object 
 7   segment           9994 non-null   object 
 8   country           9994 non-null   object 
 9   city              9994 non-null   object 
 10  state             9994 non-null   object 
 11  postal_code       9994 non-null   int64  
 12  region            9994 non-null   object 
 13  product_id        9994 non-null   object 
 14  category          9994 non-null   object 
 15  sub_category      9994 non-null   object 
 16  product_name      9994 non-null   object 


In [124]:
superstore_df.head(10)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,discount,profit,profit_margin,discounted_sales,order_year,order_month,order_day,ship_year,ship_month,ship_day
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,0.00,41.9136,0.1600,261.9600,2016,11,8,2016,11,11
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,0.00,219.5820,0.3000,731.9400,2016,11,8,2016,11,11
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,0.00,6.8714,0.4700,14.6200,2016,6,12,2016,6,16
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,0.45,-383.0310,-0.4000,526.6676,2015,10,11,2015,10,18
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,0.20,2.5164,0.1125,17.8944,2015,10,11,2015,10,18
5,6,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,0.00,14.1694,0.2900,48.8600,2014,6,9,2014,6,14
6,7,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,0.00,1.9656,0.2700,7.2800,2014,6,9,2014,6,14
7,8,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,0.20,90.7152,0.1000,725.7216,2014,6,9,2014,6,14
8,9,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,0.20,5.7825,0.3125,14.8032,2014,6,9,2014,6,14
9,10,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,0.00,34.4700,0.3000,114.9000,2014,6,9,2014,6,14


## Normalization
Break the dataset into distinct tables
- Customer
- Product
- Orders
- Sales

An integer id per table will be created for query optimization

### Customer Table
**Fields**
- customer_id (using auto increment int) [Primary Key]
- key (using customer_id)
- name
- segment
- country
- region
- state
- city
- postal_code

In [58]:
# create table
create_customer_table = """
CREATE TABLE customer (
    customer_id INT AUTO_INCREMENT,
    customer_key VARCHAR(10) UNIQUE,
    name VARCHAR(255),
    segment VARCHAR(50),
    country VARCHAR(50),
    region VARCHAR(50),
    state VARCHAR(50),
    city VARCHAR(50),
    postal_code INT,
    PRIMARY KEY (customer_id, customer_key)
);
"""
cursor.execute(create_customer_table)
connection.commit()

In [57]:
# optional: drop table if planning to modify the existing table
drop_customer_table = "DROP TABLE IF EXISTS customer;"
cursor.execute(drop_customer_table)
connection.commit()

In [59]:
# obtain unique customers
unique_customers = superstore_df.drop_duplicates(subset=['customer_id'])
unique_customers = unique_customers.where(pd.notnull(unique_customers), None)

# insert values from cleaned dataset
for index, row in unique_customers.iterrows():
    insert_customer = """
    INSERT IGNORE INTO customer (customer_key, name, segment, country, region, state, city, postal_code)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_customer, (
        row['customer_id'], row['customer_name'], row['segment'], row['country'], 
        row['region'], row['state'], row['city'], row['postal_code']
    ))

connection.commit()

# fetch and print results
select_query = "SELECT * FROM customer LIMIT 3"
cursor.execute(select_query)
result = cursor.fetchall()
for row in result:
    print(row)

(1, 'CG-12520', 'Claire Gute', 'Consumer', 'United States', 'South', 'Kentucky', 'Henderson', 42420)
(2, 'DV-13045', 'Darrin Van Huff', 'Corporate', 'United States', 'West', 'California', 'Los Angeles', 90036)
(3, 'SO-20335', "Sean O'Donnell", 'Consumer', 'United States', 'South', 'Florida', 'Fort Lauderdale', 33311)


## Product Table
**Fields**
- product_id (using auto increment int) [Primary Key]
- product_key (using product_id)
- name
- category
- sub_category

In [78]:
# create table
create_product_table = """
CREATE TABLE IF NOT EXISTS product (
    product_id INT AUTO_INCREMENT,
    product_key VARCHAR(50) UNIQUE,
    name VARCHAR(255),
    category VARCHAR(50),
    sub_category VARCHAR(50),
    PRIMARY KEY (product_id, product_key)
);
"""
cursor.execute(create_product_table)
connection.commit()

In [77]:
# optional: drop table first if planning to modify the existing table
drop_product_table = "DROP TABLE IF EXISTS product;"
cursor.execute(drop_product_table)
connection.commit()

In [79]:
# obtain unique products
unique_products = superstore_df.drop_duplicates(subset=['product_id'])
unique_products = unique_products.where(pd.notnull(unique_products), None)

# insert values from cleaned dataset
for index, row in unique_products.iterrows():
    insert_product = """
    INSERT IGNORE INTO product (product_key, name, category, sub_category)
    VALUES (%s, %s, %s, %s)
    """
    cursor.execute(insert_product, (
        row['product_id'], row['product_name'], row['category'], row['sub_category']
    ))

connection.commit()

# fetch and display results
select_query = "SELECT * FROM product LIMIT 3"
cursor.execute(select_query)
result = cursor.fetchall()
for row in result:
    print(row)

(1, 'FUR-BO-10001798', 'Bush Somerset Collection Bookcase', 'Furniture', 'Bookcases')
(2, 'FUR-CH-10000454', 'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back', 'Furniture', 'Chairs')
(3, 'OFF-LA-10000240', 'Self-Adhesive Address Labels for Typewriters by Universal', 'Office Supplies', 'Labels')


## Orders Table
**Fields**
- order_id (using auto increment int) [Primary Key]
- order_key (using order_id)
- order_date
- ship_date
- ship_mode
- order_year
- order_month
- order_day
- ship_year
- ship_month
- ship_day
- customer_id [Foreign Key]

In [81]:
# create table
create_order_table = """
CREATE TABLE IF NOT EXISTS orders (
    order_id INT AUTO_INCREMENT,
    order_key VARCHAR(50) UNIQUE,
    order_date DATE, 
    ship_date DATE,
    ship_mode VARCHAR(50),
    order_year INT,
    order_month INT,
    order_day INT,
    ship_year INT,
    ship_month INT,
    ship_day INT,
    customer_key VARCHAR(50),
    PRIMARY KEY (order_id, order_key),
    FOREIGN KEY (customer_key) REFERENCES customer(customer_key)
);
"""
cursor.execute(create_order_table)
connection.commit()

In [80]:
# optional: drop table first if planning to modify the existing table
drop_order_table = "DROP TABLE IF EXISTS orders;"
cursor.execute(drop_order_table)
connection.commit()

In [83]:
# obtain unique orders
unique_orders = superstore_df.drop_duplicates(subset=['order_id'])
unique_orders = unique_orders.where(pd.notnull(unique_orders), None)

for index, row in unique_orders.iterrows():
    insert_order = """
        INSERT IGNORE INTO orders (order_key, order_date, ship_date, ship_mode, 
        order_year, order_month, order_day, ship_year, ship_month, ship_day, customer_key)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
    cursor.execute(insert_order, (
        row['order_id'], row['order_date'], row['ship_date'], row['ship_mode'],
        row['order_year'], row['order_month'], row['order_day'],
        row['ship_year'], row['ship_month'], row['ship_day'], row['customer_id']
    ))

connection.commit()

# fetch and display results
select_query = "SELECT * FROM orders LIMIT 3"
cursor.execute(select_query)
results = cursor.fetchall()
for row in results:
    print(row)

(1, 'CA-2016-152156', datetime.date(2016, 11, 8), datetime.date(2016, 11, 11), 'Second Class', 2016, 11, 8, 2016, 11, 11, 'CG-12520')
(2, 'CA-2016-138688', datetime.date(2016, 6, 12), datetime.date(2016, 6, 16), 'Second Class', 2016, 6, 12, 2016, 6, 16, 'DV-13045')
(3, 'US-2015-108966', datetime.date(2015, 10, 11), datetime.date(2015, 10, 18), 'Standard Class', 2015, 10, 11, 2015, 10, 18, 'SO-20335')


## Sales Table
**Fields**
- sales_id (using auto increment int) [Primary Key]
- sales
- quantity 
- discount 
- profit 
- profit_margin 
- discounted_sales
- order_id [Foreign Key]
- product_id [Foreign Key]

In [90]:
# create table
create_sales_table = """
CREATE TABLE IF NOT EXISTS sales (
    sales_id INT AUTO_INCREMENT PRIMARY KEY,
    sales DECIMAL(10, 4),
    quantity INT,
    discount DECIMAL(10, 4),
    profit DECIMAL(10, 4),
    profit_margin DECIMAL(10, 4),
    discounted_sales DECIMAL(10, 4),
    order_key VARCHAR(50),
    product_key VARCHAR(50),
    FOREIGN KEY (order_key) REFERENCES orders(order_key),
    FOREIGN KEY (product_key) REFERENCES product(product_key)
);
"""
cursor.execute(create_sales_table)
connection.commit()

In [89]:
# optional: drop table first if planning to modify the existing table
drop_sales_table = "DROP TABLE IF EXISTS sales;"
cursor.execute(drop_sales_table)
connection.commit()

In [97]:
# insert values from cleaned dataset
for index, row in superstore_df.iterrows():
    insert_sales = """
    INSERT IGNORE INTO sales (sales, quantity, discount, profit, profit_margin, discounted_sales, order_key, product_key)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_sales, (
        row['sales'], row['quantity'], row['discount'], row['profit'], 
        row['profit_margin'], row['discounted_sales'], row['order_id'], row['product_id']
    ))

connection.commit()

# fetch all display results
select_query = "SELECT * FROM sales LIMIT 10"
cursor.execute(select_query)
results = cursor.fetchall()
for row in results:
    print(row)

(1, Decimal('261.9600'), 2, Decimal('0.0000'), Decimal('41.9136'), Decimal('0.1600'), Decimal('261.9600'), 'CA-2016-152156', 'FUR-BO-10001798')
(2, Decimal('731.9400'), 3, Decimal('0.0000'), Decimal('219.5820'), Decimal('0.3000'), Decimal('731.9400'), 'CA-2016-152156', 'FUR-CH-10000454')
(3, Decimal('14.6200'), 2, Decimal('0.0000'), Decimal('6.8714'), Decimal('0.4700'), Decimal('14.6200'), 'CA-2016-138688', 'OFF-LA-10000240')
(4, Decimal('957.5775'), 5, Decimal('0.4500'), Decimal('-383.0310'), Decimal('-0.4000'), Decimal('526.6676'), 'US-2015-108966', 'FUR-TA-10000577')
(5, Decimal('22.3680'), 2, Decimal('0.2000'), Decimal('2.5164'), Decimal('0.1125'), Decimal('17.8944'), 'US-2015-108966', 'OFF-ST-10000760')
(6, Decimal('48.8600'), 7, Decimal('0.0000'), Decimal('14.1694'), Decimal('0.2900'), Decimal('48.8600'), 'CA-2014-115812', 'FUR-FU-10001487')
(7, Decimal('7.2800'), 4, Decimal('0.0000'), Decimal('1.9656'), Decimal('0.2700'), Decimal('7.2800'), 'CA-2014-115812', 'OFF-AR-10002833')
(

## Superstore Sales Table (all combined)

In [110]:
# create table
create_superstore_table = """
CREATE TABLE IF NOT EXISTS superstore (
    sales_id INT PRIMARY KEY,
    customer_key VARCHAR(10),
    customer_name VARCHAR(255),
    segment VARCHAR(50),
    country VARCHAR(50),
    region VARCHAR(50),
    state VARCHAR(50),
    city VARCHAR(50),
    postal_code INT,
    product_key VARCHAR(50),
    product_name VARCHAR(255),
    category VARCHAR(50),
    sub_category VARCHAR(50),
    order_key VARCHAR(50),
    order_date DATE, 
    ship_date DATE,
    ship_mode VARCHAR(50),
    order_year INT,
    order_month INT,
    order_day INT,
    ship_year INT,
    ship_month INT,
    ship_day INT,
    sales DECIMAL(10, 4),
    quantity INT,
    discount DECIMAL(10, 4),
    profit DECIMAL(10, 4),
    profit_margin DECIMAL(10, 4),
    discounted_sales DECIMAL(10, 4),
    FOREIGN KEY (customer_key) REFERENCES customer(customer_key),
    FOREIGN KEY (order_key) REFERENCES orders(order_key),
    FOREIGN KEY (product_key) REFERENCES product(product_key)
);
"""
cursor.execute(create_superstore_table)
connection.commit()

In [109]:
# optional: drop table first if planning to modify the existing table
drop_superstore_table = "DROP TABLE IF EXISTS superstore;"
cursor.execute(drop_superstore_table)
connection.commit()

In [125]:
# insert values from cleaned dataset
for index, row in superstore_df.iterrows():
    insert_superstore = """
    INSERT IGNORE INTO superstore (sales_id, customer_key, customer_name, segment, country, region, state, city, postal_code, 
    product_key, product_name, category, sub_category, 
    order_key, order_date, ship_date, ship_mode, order_year, order_month, order_day, ship_year, ship_month, ship_day, 
    sales, quantity, discount, profit, profit_margin, discounted_sales)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_superstore, (
        row['row_id'], row['customer_id'], row['customer_name'], row['segment'], row['country'], row['region'], row['state'], row['city'], row['postal_code'],
        row['product_id'], row['product_name'], row['category'], row['sub_category'],
        row['order_id'], row['order_date'], row['ship_date'], row['ship_mode'], row['order_year'], row['order_month'], row['order_day'], row['ship_year'], row['ship_month'], row['ship_day'],
        row['sales'], row['quantity'], row['discount'], row['profit'], row['profit_margin'], row['discounted_sales']
    ))

connection.commit()

# fetch and display results
select_query = "SELECT * FROM superstore LIMIT 10"
cursor.execute(select_query)
results = cursor.fetchall()
df = pd.DataFrame(results)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,1,CG-12520,Claire Gute,Consumer,United States,South,Kentucky,Henderson,42420,FUR-BO-10001798,...,8,2016,11,11,261.9600,2,0.0000,41.9136,0.1600,261.9600
1,2,CG-12520,Claire Gute,Consumer,United States,South,Kentucky,Henderson,42420,FUR-CH-10000454,...,8,2016,11,11,731.9400,3,0.0000,219.5820,0.3000,731.9400
2,3,DV-13045,Darrin Van Huff,Corporate,United States,West,California,Los Angeles,90036,OFF-LA-10000240,...,12,2016,6,16,14.6200,2,0.0000,6.8714,0.4700,14.6200
3,4,SO-20335,Sean O'Donnell,Consumer,United States,South,Florida,Fort Lauderdale,33311,FUR-TA-10000577,...,11,2015,10,18,957.5775,5,0.4500,-383.0310,-0.4000,526.6676
4,5,SO-20335,Sean O'Donnell,Consumer,United States,South,Florida,Fort Lauderdale,33311,OFF-ST-10000760,...,11,2015,10,18,22.3680,2,0.2000,2.5164,0.1125,17.8944
